In [1]:
# Importing all necessary libraries
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
 
img_width, img_height = 224, 224

In [2]:
train_data_dir = '../input/imageclasscification/vdata/train'
validation_data_dir = '../input/imageclasscification/vdata/test'
nb_train_samples =400
nb_validation_samples = 100
epochs = 10
batch_size = 16

In [3]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [4]:
model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

2022-05-24 10:58:38.434154: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [5]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
 
test_datagen = ImageDataGenerator(rescale=1. / 255)
 
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')
 
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')
 
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Found 400 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-05-24 10:58:39.677512: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
25/25 [==============================] - 12s 421ms/step - loss: 1.1489 - accuracy: 0.5900 - val_loss: 0.5817 - val_accuracy: 0.8750
Epoch 2/10
25/25 [==============================] - 9s 372ms/step - loss: 0.5870 - accuracy: 0.7325 - val_loss: 0.3383 - val_accuracy: 0.8854
Epoch 3/10
25/25 [==============================] - 9s 370ms/step - loss: 0.4974 - accuracy: 0.8025 - val_loss: 0.4359 - val_accuracy: 0.7604
Epoch 4/10
25/25 [==============================] - 10s 393ms/step - loss: 0.4315 - accuracy: 0.8050 - val_loss: 0.6138 - val_accuracy: 0.6146
Epoch 5/10
25/25 [==============================] - 9s 368ms/step - loss: 0.4418 - accuracy: 0.8275 - val_loss: 0.2728 - val_accuracy: 0.8854
Epoch 6/10
25/25 [==============================] - 9s 371ms/step - loss: 0.3774 - accuracy: 0.8725 - val_loss: 0.3223 - val_accuracy: 0.8333
Epoch 7/10
25/25 [==============================] - 10s 380ms/step - loss: 0.3815 - accuracy: 0.8675 - val_loss: 0.3009 - val_accuracy: 0.8646
Epo

In [7]:
model.save_weights('model_saved.h5')

In [8]:
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
import numpy as np
 
from keras.models import load_model
 
#model.load_weights('./model_saved.h5')
 
image = load_img('../input/imageclasscification/vdata/test/cars/13.jpg', target_size=(224, 224))
img = np.array(image)
img = img / 255.0
img = img.reshape(1,224,224,3)
label = model.predict(img)
print("Predicted Class (0 - Cars , 1- Planes): ", label[0][0])

Predicted Class (0 - Cars , 1- Planes):  0.09850678


In [9]:
result = model.evaluate(validation_generator)

7/7 [==============================] - 0s 59ms/step - loss: 0.2829 - accuracy: 0.8800
